In [216]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
data_mat = scipy.io.loadmat('ex4data1.mat')
weight_mat = scipy.io.loadmat('ex4weights.mat')
data_mat['X'].shape, data_mat['y'].shape, weight_mat['Theta1'].T.shape, weight_mat['Theta2'].T.shape

((5000, 400), (5000, 1), (401, 25), (26, 10))

In [217]:
X = data_mat['X']
y = data_mat['y']
theta1 = weight_mat['Theta1'].T
theta2 = weight_mat['Theta2'].T

In [500]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

# Feedforward Propagation and Prediction
# args: theta need be transposed, theta shape: (L-1层特征值+1， L层特征值)
# call: layer(x, theta)
# out shape: (m, layer_size), like: a1.shape(5000, 400), a2.shape(5000, 25), a3.shape(5000, 10)
def layer_no_zero(x, theta):
    m = x.shape[0]
    num_labels = theta.shape[0]
    layer_input = np.append(np.ones((m, 1)), x, axis=1)
    z = np.dot(layer_input, theta)
    layer_out = sigmoid(z)
    return layer_out

# a1.shape(5000, 400), a2.shape(5000, 25), a3.shape(5000, 10)
def forward(Theta1, Theta2, X):
    a1 = X
    a2 = layer_no_zero(a1, Theta1)
    a3 = layer_no_zero(a2, Theta2)
    return a1, a2, a3

# shape(5000, 10)
# num_labels 为分类的数字（1-10）
def transfer_y(y, num_labels):
    m = y.size
    I = np.eye(num_labels)
    Y = np.zeros((m, num_labels))
    for i in np.arange(m):
        Y[i, :] = I[y[i][0] - 1, :]
    return Y

def cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_nn):
    
    Theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
    Theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')
    
    m = X.shape[0]
    Y = transfer_y(y, num_labels)
    a1, a2, h = forward(Theta1, Theta2, X)

    regulariz_first_layer = np.sum(np.power(Theta1[1:, :], 2))
    regulariz_second_layer = np.sum(np.power(Theta2[1:, :], 2))
    regulariz = (lambda_nn/(2*m))*(regulariz_first_layer + regulariz_second_layer)
    
    # 点乘(*, 对应位置相乘，两个乘数和结果的shape一样)，不是向量(np.dot)乘积
    J = (1.0/m) * np.sum((-Y) * np.log(h) - (1-Y) * np.log(1-h))
    J = J + regulariz
    return J

# a_1 = (5000, 401)
# a_2 = (5000, 26)
# a_3 = (5000, 10)
# theta_1 = (401, 25)
# theta_2 = (26, 10)
# d_1 = (401, 25)
# d_2 = (26, 10)
# d_3 = (5000, 10)
def grad_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_nn):
    
    Theta1 = np.reshape(nn_params[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
    Theta2 = np.reshape(nn_params[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')
    
    # a1.shape(5000, 400), a2.shape(5000, 25), a3.shape(5000, 10)
    a1, a2, a3 = forward(Theta1, Theta2, X)
    a1_full = np.append(np.ones((a1.shape[0], 1)), a1, axis=1)
    a2_full = np.append(np.ones((a2.shape[0], 1)), a2, axis=1)
    
    # Delta3.shape(5000, 10)
    Y = transfer_y(y, num_labels)
    Delta3 = a3 - Y
    # Delta2.shape(26, 10) = a2_full.T.shape(26, 5000) * d3.shape(5000, 10)
    Delta2 = np.dot(a2_full.T, Delta3)
    # d2.shape(5000, 25) = (5000, 10) * (26-1, 10).T .* (5000, 25) .* (5000, 25)
    d2 = np.dot(Delta3, Theta2[1:, :].T) * a2 * (1-a2)
    # Delta1.shape(401, 25) = a1_full.T.shape(401, 5000) * d2.shape(5000, 25)
    Delta1 = np.dot(a1_full.T, d2)
    
    Theta2_new = 1/m * Delta2 + np.append(np.zeros((1, Theta2.shape[1])), lambda_nn/m * Theta2[1:, :], axis=0)
    Theta1_new = 1/m * Delta1 + np.append(np.zeros((1, Theta1.shape[1])), lambda_nn/m * Theta1[1:, :], axis=0)
        
    return np.append(Theta1_new.flatten(), Theta2_new.flatten())

In [501]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
nn_params = np.append(theta1.flatten(), theta2.flatten())
cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 1)

0.38376985909092365

In [503]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
nn_params = np.append(theta1.flatten(), theta2.flatten())
Theta_new = grad_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 1)
Theta_new.shape

(10285,)

In [504]:
import scipy.optimize as op

input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
nn_params = np.append(theta1.flatten(), theta2.flatten())
# x0 must be（n,0）
result_reg = op.minimize(fun = cost_function, x0 = nn_params, args = (input_layer_size, hidden_layer_size, num_labels, X, y, 1), method = 'CG', jac = grad_function)

nn_theta_reg = result_reg.x
result_reg


     fun: 0.301214620603095
     jac: array([-3.65227123e-06,  8.98240852e-07, -5.59367396e-06, ...,
       -9.19724902e-08,  9.38889915e-07, -3.39321657e-07])
 message: 'Optimization terminated successfully.'
    nfev: 6520
     nit: 2693
    njev: 6520
  status: 0
 success: True
       x: array([ 1.22650776, -2.17616782,  1.91865553, ..., -2.67391977,
       -2.70700525, -2.09755106])

In [505]:
# test for scipy.optimize

input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
Theta1_final_nn = np.reshape(nn_theta_reg[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
Theta2_final_nn = np.reshape(nn_theta_reg[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')

a1, a2, a3 = forward(Theta1_final_nn, Theta2_final_nn, X)
pred = np.argmax(a3, axis = 1) + 1
np.mean(pred == y.flatten()) * 100


99.68

In [481]:
# use for loop to decrease grad
# https://github.com/Benlau93/Machine-Learning-by-Andrew-Ng-in-Python/tree/master/LogisticRegression

def gradientDescentnn(X,y,initial_nn_params,alpha,num_iters,Lambda,input_layer_size, hidden_layer_size, num_labels):
    
    Theta1 = np.reshape(initial_nn_params[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
    Theta2 = np.reshape(initial_nn_params[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')
    
    J_history =[]
    
    for i in range(num_iters):
        nn_params = np.append(Theta1.flatten(),Theta2.flatten())
        cost = cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 1)
        grad = grad_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, transfer_y(y, num_labels), 0)
        
        grad1 = np.reshape(grad[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
        grad2 = np.reshape(grad[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')
    
        Theta1 = Theta1 - (alpha * grad1)
        Theta2 = Theta2 - (alpha * grad2)
        J_history.append(cost)
    
    nn_paramsFinal = np.append(Theta1.flatten(),Theta2.flatten())
    return nn_paramsFinal , J_history


In [483]:
def randInitializeWeights(L_in, L_out):
    epsilon = 0.12
    return np.random.rand(L_in+1, L_out) * 2 * epsilon - epsilon

input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
initial_theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_theta2 = randInitializeWeights(hidden_layer_size, num_labels)
initial_nn_params = np.append(initial_theta1.flatten(), initial_theta2.flatten())
nn_paramsFinal, nnJ_history = gradientDescentnn(X,y,initial_nn_params,0.8,300,1,input_layer_size, hidden_layer_size, num_labels)

# Theta1_final = np.reshape(nn_paramsFinal[:hidden_layer_size*(input_layer_size+1)], (input_layer_size+1, hidden_layer_size), 'C')
# Theta2_final = np.reshape(nn_paramsFinal[hidden_layer_size*(input_layer_size+1):], (hidden_layer_size+1, num_labels), 'C')
nnJ_history[-5:]

[0.7534771457290492,
 0.7522031518744885,
 0.7509379228342219,
 0.7496813739301598,
 0.7484334213570907]

In [484]:
# test for loop
a1, a2, a3 = forward(Theta1_final, Theta2_final, X)
pred = np.argmax(a3, axis = 1) + 1
np.mean(pred == y.flatten()) * 100

97.61999999999999

(5000, 10)